In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import resnet
import numpy as np

from datetime import datetime
import torchvision.models as models
from networks.vgg16 import VGG

import sys 
sys.path.append('..')
from utils_gsp.logger import Logger
from utils_gsp import sps_tools
from gsp_model import GSP_Model
import tools.ipynb_funcs as utilfuncs
import tools.visualization as viz

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from main import train, validate, accuracy, save_checkpoint, setup_experiment, gsp_sparse_training, AverageMeter

model_names = sorted(name for name in resnet.__dict__
    if name.islower() and not name.startswith("__")
                     and name.startswith("resnet")
                     and callable(resnet.__dict__[name]))

print(model_names)

['resnet110', 'resnet1202', 'resnet20', 'resnet32', 'resnet44', 'resnet56']
['resnet110', 'resnet1202', 'resnet20', 'resnet32', 'resnet44', 'resnet56']


In [3]:
class Args:
    arch = 'resnet20'
    workers = 4
    epochs=160
    start_epoch=0
    batch_size = 128
    lr = 0.1
    momentum = 0.9
    weight_decay=1e-4
    print_freq = 50
    resume = False
    evaluate = False
    pretrained = False
    half = False
    exp_name = 'gsp_test'

    gpu=None
    logdir = '/logdir'
    gsp_training = True 
    gsp_sps = 0.8
    gsp_int = 150
    gsp_start_ep = -1
    finetune = False
    finetune_sps = 0.9


global args, best_acc1
args = Args
# writer = SummaryWriter()

In [ ]:
# args.resume = './results/gspS0.9/gsp/model_best.pth.tar' # LOAD GSP MODEL

# args.resume = './results/gspS0.80/fine_0.9/model_best.pth.tar'
# args.finetune = True

In [10]:
# torch.manual_seed(0)
# Setup the experiment
flogger = setup_experiment(args)
args.logger.log_cmd_arguments(args)

# model = torch.nn.DataParallel(resnet.__dict__[args.arch]())
# model = VGG('D')

model.cuda()

model_gsp = GSP_Model(model)

if args.finetune:
    flogger.info(15*"*" + " Model will be finetuned!! " + 15*"*")
    model_gsp.prune_and_mask_model(sps=args.finetune_sps)
    
# optionally resume from a checkpoint
if args.resume:
    if os.path.isfile(args.resume):
        print("=> loading checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        args.start_epoch = checkpoint['epoch']
        best_acc1 = checkpoint['best_acc1']
        model_gsp.model.load_state_dict(checkpoint['state_dict'])
        print("=> loaded checkpoint '{}' (epoch {})"
                .format(args.evaluate, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(args.resume))

cudnn.benchmark = True

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='./data', train=True, transform=transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, 4),
        transforms.ToTensor(),
        normalize,
    ]), download=True),
    batch_size=args.batch_size, shuffle=True,
    num_workers=args.workers, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=128, shuffle=False,
    num_workers=args.workers, pin_memory=True)


# ----------------------- Make a GSP Model -----------------------
print(f"The sparsity of the model is: {model_gsp.get_model_sps():.2f}")
args.writer = SummaryWriter(log_dir=f'results/{args.exp_name}/runs/{datetime.now().strftime("%m-%d_%H:%M")}')

# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum,
                            weight_decay=args.weight_decay)

# ============================ Setup GSP model ============================
if args.gsp_training:
    gsp_sparse_training(model_gsp, train_loader, args)
    flogger.info(15*"*" + " Model will be trained with GSP Sparsity!! " + 15*"*" )


 input logdir: ./results/gsp_test/logdir 


 RETURNED FILE LOGGER! 

Files already downloaded and verified
The sparsity of the model is: 0.03


In [11]:

train(train_loader, model_gsp, criterion, optimizer, 0, args, gsp_mode=False)
# validate(val_loader, model, criterion, args)

/home/users/rohib/anaconda3/envs/imagenet/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


NameError: name 'F' is not defined

In [ ]:
model_gsp.get_model_sps()

In [ ]:
def get_model_sps(self):
    nonzero = total = 0
    for name, param in self.model.named_parameters():
        if 'mask' not in name:
            tensor = param.detach().clone()
            # nz_count.append(torch.count_nonzero(tensor))
            nz_count = torch.count_nonzero(tensor).item()
            total_params = tensor.numel()
            nonzero += nz_count
            total += total_params
    
    tensor = None
    # print(f"TOTAL: {total}")
    abs_sps = 100 * (total-nonzero) / total
    return abs_sps

In [ ]:
get_model_sps(model_gsp)

In [ ]:
model_gsp.force_apply_gsp(0.90)

In [ ]:
model_gsp.prune_and_mask_model(sps=0.9)

## MODEL VISUALIZATION

In [ ]:
names, weight_l, shape_l, weight_d, layers_d = utilfuncs.get_model_layers(model)

In [ ]:
# viz.plot_weight_dist(model, title='Resnet20 Distribution', arch='resnet20', 
#                     filename='./results/gspS0.80/fine_0.9/w_dist.png')

In [ ]:
def apply_gsp_prune(model_gsp, sps=0.8):
    model_gsp.force_apply_gsp(sps)
    print(f"SPS after gsp application: {model_gsp.get_model_sps():.2f}")
    
    model_gsp.prune_and_mask_model(sps)
    print(f"SPS after pruning: {model_gsp.get_model_sps():.2f}")


In [ ]:
apply_gsp_prune(model_gsp, sps=0.9)